## 라이브러리

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# layers 에서는 Conv2D, MaxPool2D, Dropout, Flatten, Dense 를 불러옵니다.
# callbacks 에서는 EarlyStopping 을 불러옵니다.
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

## 이미지 폴더 보기

In [ ]:
# 이미지 다운로드
!wget https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip

In [ ]:
# images 폴더에 다운로드 받은 파일 압축 해제하기
!unzip cell_images.zip

In [ ]:
import os
for root, dirs, files in os.walk("./cell_images/"):
    print(root, dirs, len(files))

## 일부 이미지 미리보기

In [ ]:
import glob
upics = glob.glob('./cell_images/Uninfected/*.png')
apics = glob.glob('./cell_images/Parasitized/*.png')
len(upics), upics[0], len(apics), apics[0]

In [ ]:
# upics 
upics_0 = upics[0]
upics_0_img = plt.imread(upics_0)
plt.imshow(upics_0_img)

In [ ]:
# apics

apics_0 = apics[0]
apics_0_img = plt.imread(apics_0)
plt.imshow(apics_0_img)

In [ ]:
# cv2 로 Uninfected 시각화
import cv2

plt.figure(figsize=(8, 8))
labels = "Uninfected"
for i, images in enumerate(upics[:9]):
    ax = plt.subplot(3, 3, i + 1)
    img = cv2.imread(images)
    plt.imshow(img)
    plt.title(f'{labels} {img.shape}')
    plt.axis("off")

In [ ]:
# cv2 로 Infected 시각화
plt.figure(figsize=(8, 8))
labels = "Infected"

for i, images in enumerate(apics[:9]):
    ax = plt.subplot(3, 3, i + 1)
    img = cv2.imread(images)
    plt.imshow(img)
    plt.title(f'{labels} {img.shape}')
    plt.axis("off")


## 데이터셋 나누기
학습, 검증 세트

In [ ]:
# ImageDataGenerator 를 통해 이미지를 로드하고 전처리 합니다.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# validation_split 값을 통해 학습:검증 비율을 8:2 로 나눕니다.
datagen = ImageDataGenerator(rescale=1/255.0, validation_split=0.2)
datagen

## 이미지 사이즈 설정
이미지의 사이즈가 불규칙하면 학습을 할 수 없기 때문에

이미지 사이즈를 재지정

In [ ]:
# 원본 이미지는 100~200 내외입니다.
width = 64
height = 64

학습 세트 (train)

In [ ]:
# flow_from_directory 를 통해 이미지를 불러옵니다.
# training 데이터셋을 생성합니다.
# class_mode 에는 이진분류이기 때문에 binary 를 넣어줍니다.
trainDatagen = datagen.flow_from_directory(directory = 'cell_images/',
                                           target_size = (height, width),
                                           class_mode = 'binary',
                                           batch_size = 64,
                                           subset='training')

In [ ]:
trainDatagen.num_classes

In [ ]:
trainDatagen.classes

검증 세트 (test)

In [ ]:
# validation 데이터셋을 생성합니다.
# class_mode: One of "categorical", "binary", "sparse",
#     "input", or None. Default: "categorical".
# subset: Subset of data ("training" or "validation")
valDatagen = datagen.flow_from_directory(directory = 'cell_images/',
                                         target_size =(height, width),
                                         class_mode = 'binary',
                                         batch_size = 64,
                                         subset='validation')

In [ ]:
# {'Parasitized': 0, 'Uninfected': 1}
# 0 : 감염, 1: 감염 안 됨
valDatagen.class_indices

## 레이어 설정

CNN에서 사용되는 대표적인 레이어: Conv2D, MaxPolling2D

* Conv2D
    * 2D 합성곱 연산을 수행하는 레이어
    * 입력 데이터를 필터와 함께 함성곱하여 출력 데이터를 생성
    * 필터는 입력 데이터의 부분 영역과 요소곱을 수행하고 그 결과를 모두 더하여 출력 데이터의 한 요소를 생성
    * 이때 필터를 슬라이딩해가며 입력 데이터 전체에 대해 합성곱을 수행
    * 일반적으로 여러 개의 필터를 사용하여 여러 개의 출력 데이터 (Feature Map)을 생성
    * 이렇게 생성된 특징 맵은 다음 레이어로 전달되어 더 복잡한 특징을 추출하는 데에 사용

* MaxPool2D
    * 2D 맥스 풀링 연산을 수행하는 레이어
    * 입력 데이터의 영역을 나누고, 각 영역에서 가장 큰 값을 선택하여 출력 데이터를 생성
    * 이를 통해 입력 데이터의 크기를 줄이고, 필터링 효과를 얻을 수 있음
    * 주로 Conv2D 레이어 다음에 위치하여 특징 맵의 크기를 줄이는 데 사용

* filter
    * 컨볼루션의 필터 수
    * 특징 맵 수

* kernel_size
    * 컨볼루션 커널의 행, 열
    * 필터 사이즈

* padding
    * 경계 처리 방법
    * 'valid': 유효한 영역만 출력. 따라서 출력 이미지 사이즈는 입력 사이즈보다 작음
    * 'same': 출력 이미지 사이즈 = 입력 이미지 사이즈

* input_shape
    * 모델에서 첫 레이어에 정의
    * (height, width, channels)

* activation
    * 활성화 함수 설정
    * 'linear': 기본값, 입력 뉴런과 가중치로 계산된 결과값이 그대로 출력
    * 'relu': rectifier 함수, 은닉층에 주로 사용
    * 'sigmoid': 시그모이드 함수, 이진 분류 문제에서 출력층에 주로 사용 (예측값이 둘 중 하나)
    * 'softmax': 소프트맥스 함수, 다중 클래스 분류 문제에서 출력층에 주로 사용 (예측값이 n개)

In [ ]:
from tensorflow.keras.layers import MaxPooling2D

In [ ]:
# padding : 경계 처리 방법
# ‘valid’ : 유효한 영역만 출력이 됩니다. 따라서 출력 이미지 사이즈는 입력 사이즈보다 작습니다.
# ‘same’ : 출력 이미지 사이즈가 입력 이미지 사이즈와 동일합니다.
model = Sequential()
# 입력층
model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(height, width, 3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Fully-connected layer
model.add(Flatten())
model.add(Dense(32, activation='relu'))

# 출력층 => 말라리아 감염여부를 분류합니다.
model.add(Dense(1, activation='sigmoid'))

모델 요약

In [ ]:
# summary
model.summary()

In [ ]:
# tensorflow.keras.utils의 plot_model 을 통한 레이어 시각화

from tensorflow.keras.utils import plot_model

plot_model(model)

## 컴파일

* Optimizer
    * 데이터와 손실 함수를 바탕으로 모델의 업데이트 방법을 결정

* Metrics
    * 훈련 단계와 테스트 단계를 모니터링하기 위해 사용

* Loss function (손실 함수)
    - 훈련하는 동안 모델의 오차를 측정
    - 모델의 학습이 올바른 방향으로 향하도록 이 함수를 최소화해야함
    - 최적의 가중치를 찾도록 해야함

    - 회귀: MSE, MAE
    - 분류
        - 바이너리: binary_crossentropy
        - 멀티클래스: categorical_crossentropy, sparse_categorical_crossentropy

In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-3)
optim

In [ ]:
# model.compile
# 옵티마이저 'adam'
# 손실함수 이진분류
# 측정지표 'accuracy'

model.compile(optimizer=optim,
             loss=tf.keras.losses.BinaryCrossentropy(),
             metrics=['accuracy'])

## 학습

* batch: 모델 학습에 한 번에 입력할 데이터셋
* epoch: 모델 학습 시 전체 데이터를 학습한 횟수
* step: 하나의 배치를 학습한 횟수
* EarlyStopping: 성능이 더 이상 좋아지지 않으면 학습을 중지

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=2)
early_stop

In [ ]:
# fit

history = model.fit(trainDatagen, epochs=1000,
                   callbacks=early_stop,
                   validation_data=valDatagen)

In [ ]:
# history

history 